In [43]:
%pip install "git+https://github.com/jemisjoky/TorchMPS.git"
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
%pip install torchmetrics
from torchmetrics.classification import MulticlassAccuracy
from torchmps import MPS

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/jemisjoky/TorchMPS.git to /tmp/pip-req-build-5erqxi9v
  Running command git clone --filter=blob:none --quiet https://github.com/jemisjoky/TorchMPS.git /tmp/pip-req-build-5erqxi9v
  Resolved https://github.com/jemisjoky/TorchMPS.git to commit 6c0bc1a8e2c15acba8570ca9ffe2b4a0c7135165
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Hyperparameters

In [44]:
# Hardware hyperparameters
chosen_device = torch.device('cuda' 
if torch.cuda.is_available() else 'cpu')

# Data hyperparameters
nb_train_HP = 200
nb_test_HP = 50
batch_sz_HP = 150
batch_sz_HP = min(batch_sz_HP, nb_train_HP)
nb_classes_HP = 10

# Teacher hyperparameters
nteacher_epochs_HP = 25
hidden_size_HP = 70
teacher_loss_HP = nn.CrossEntropyLoss()
#Optimizer parameters
teacher_lr_HP = 0.01

# Student hyperparameters
# MPS parameters
bond_dim_HP = 20
adaptive_mode_HP = False
periodic_bc_HP = False
feature_map_HP = lambda x : torch.tensor([1, x]).to(chosen_device)

# Training parameters
ngauss_epochs_HP = 5 # number of epochs with added gaussian noise
nepochs_student_HP = 25 + ngauss_epochs_HP
student_lr_HP = 1e-3
student_reg_HP = 0.01
student_loss_HP = nn.KLDivLoss(reduction="batchmean", log_target=True)
# Gaussian parameters
gn_var_HP = 0.3 #added gaussian noise variance
gn_mean_HP = 0 #added gaussian noise mean
# well suited loss for comparison of divergences

# Premilinaries: Importing the data and utils subroutines

In [45]:
# Import the mnist train dataset
train_set = torchvision.datasets.MNIST(
    root = './datasets', train = True,   
    transform = transforms.ToTensor(),  download = True )

# Create a training batch iterator
train_subset = torch.utils.data.SubsetRandomSampler(range(nb_train_HP))
train_iterator = torch.utils.data.DataLoader(
    dataset = train_set, 
    sampler = train_subset, batch_size=batch_sz_HP
    )

# Import the mnist test set
test_set = torchvision.datasets.MNIST(
    root = './datasets',
    train = False, transform = transforms.ToTensor(),  download = True
    )
# Create a testing batch iterator
test_subset = torch.utils.data.SubsetRandomSampler(range(nb_test_HP))
test_iterator = torch.utils.data.DataLoader(
    dataset = test_set, 
    sampler = test_subset, batch_size = batch_sz_HP
    )

In [46]:
# Returns the validation set classification accuracy
# of the given input model (this is a higher order function)
def get_acc(model, iterator):
    # Get the validation set classification accuracy
    total_good_classifications = 0
    acc_metric = MulticlassAccuracy(num_classes=nb_classes_HP).to(chosen_device)
    for (x_mb, y_mb) in iterator:
        x_mb = x_mb.reshape(-1, 784).to(chosen_device)
        y_mb = y_mb.to(chosen_device)
        # Add the number of datapoints we classified right to the total
        batch_size = x_mb.size()[0]
        y_hat = model(x_mb)
        batch_good_classifications = batch_size * acc_metric(y_hat, y_mb)
        total_good_classifications += batch_good_classifications
    return total_good_classifications / nb_test_HP # divide by total size

# Training the teacher model

In [47]:

# Create the fcnn class
class FCNN(nn.Module):
    def __init__(self):
        super(FCNN, self).__init__()
        self.relu = nn.ReLU()
        self.lin1 = nn.Linear(784, hidden_size_HP)
        self.lin2 = nn.Linear(hidden_size_HP, hidden_size_HP)
        self.lin3 = nn.Linear(hidden_size_HP, hidden_size_HP)
        self.lin4 = nn.Linear(hidden_size_HP, hidden_size_HP)
        self.lin5 = nn.Linear(hidden_size_HP, hidden_size_HP)
        self.lin6 = nn.Linear(hidden_size_HP, 10)

    def forward(self, x):
        y = self.lin1(x)
        y = self.relu(y)
        y = self.lin2(y)
        y = self.relu(y)
        y = self.lin3(y)
        y = self.relu(y)
        y = self.lin4(y)
        y = self.relu(y)
        y = self.lin5(y)
        y = self.relu(y)
        y = self.lin6(y)
        y = self.relu(y)
        return y

#Instantiate and put the model on the chosen device
teacher = FCNN().to(chosen_device)

#Instantiate the optimizer
teacher_optimizer = torch.optim.Adam(teacher.parameters())

# Create an array to store the val loss
# of the teacher at each epoch
teacher_val_loss = []

#Training loop
for epoch in range(nteacher_epochs_HP):
    for (x_mb, y_mb) in train_iterator:
        # Reshape the train_tuple and put on the chosen device
        x_mb = x_mb.reshape(-1, 784).to(chosen_device)
        y_mb = y_mb.to(chosen_device)
        # Foward propagation
        y_hat_mb = teacher(x_mb)
        # Backpropagation
        teacher_loss_HP(y_hat_mb, y_mb).backward()
        teacher_optimizer.step()
        teacher_optimizer.zero_grad()
        
    val_score = get_acc(teacher, test_iterator).item()
    val_score = round(val_score, 3)
    teacher_val_loss.append( val_score )

print("Teacher accuracy through epochs")
print(np.arange(1, nteacher_epochs_HP+1).tolist())
print(teacher_val_loss)


Teacher accuracy through epochs
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.18, 0.18, 0.222, 0.222, 0.244, 0.238, 0.238, 0.258, 0.238, 0.258, 0.198, 0.226, 0.299, 0.357, 0.297, 0.348, 0.376, 0.419, 0.419, 0.427]


# Training the student model

In [48]:

# Initialize the MPS module
student = MPS(
    input_dim = 28 ** 2,
    output_dim = 10,
    bond_dim = bond_dim_HP,
    adaptive_mode = adaptive_mode_HP,
    periodic_bc = periodic_bc_HP,
).to(chosen_device)
student.register_feature_map(feature_map_HP)

# Instantiate the optimizer and softmax
student_optimizer = torch.optim.Adam(
    student.parameters(), lr = student_lr_HP, weight_decay = student_reg_HP
)

# Used on the inputs before the loss function
LogSoftmax = nn.LogSoftmax(dim=1)

# Create an array to store the val loss
# of the student at each epoch
stud_test_loss = []
stud_train_loss = []

# Training loop 
for epoch in range(nepochs_student_HP):

    for (x_mb, _) in train_iterator:
        
        # Flatten the MNIST images, which come in matrix form
        x_mb = x_mb.reshape(-1, 784).to(chosen_device)

        # Apply Gaussian noise for the gaussian noised epochs
        if epoch > (nepochs_student_HP - ngauss_epochs_HP):
          x_mb = x_mb + torch.randn(size=x_mb.size()).to(chosen_device)

        # Get log of softmax of the teacher and student logits
        # required before passing to KL divergence loss in Pytorch for some reason
        teacher_output = LogSoftmax(teacher(x_mb))
        student_output = LogSoftmax(student(x_mb))

        # Backpropagation
        loss = student_loss_HP(student_output, teacher_output)
        loss.backward()
        student_optimizer.step()
        student_optimizer.zero_grad()
    #stud_train_loss.append( round(get_acc(student, train_iterator).item(), 3) )
    stud_test_loss.append( round(get_acc(student, test_iterator).item(), 3) )
    print(stud_test_loss)
    print(stud_train_loss)

print("Epochs: ", np.arange(1, nepochs_student_HP+1).tolist())
print("Train loss: ", stud_train_loss)
print("Test loss: ", stud_test_loss)


[0.1]
[]
[0.1, 0.1]
[]
[0.1, 0.1, 0.1]
[]
[0.1, 0.1, 0.1, 0.1]
[]


KeyboardInterrupt: ignored